In [1]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

Setup Complete. Use CLIENT to access Weaviate.


In [2]:
class_name = "Work"
count = CLIENT.query.aggregate(class_name).with_meta_count().do()
jprint(count)

<IPython.core.display.JSON object>

In [3]:
jprint(
    CLIENT.query
        .get(class_name="Work", properties=[])
        .with_limit(20)
        .with_near_text({"concepts": ["pictures of musicians"]})
        .with_group_by(["collection"], groups=10, objects_per_group=2)
        .with_additional([
            """
            group {
                id
                groupedBy {
                    value
                }
                count
                hits {
                    identifier
                    source
                    title
                    subject
                }
            }
            """
        ])
        .do()
)

<IPython.core.display.JSON object>

In [3]:
work=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title", "collection"])
        .with_limit(1)
        .with_additional(["vector"])
        .do()
)["data"]["Get"]["Work"][0]
vector=work["_additional"]["vector"]
jprint(work, expanded=False)

<IPython.core.display.JSON object>

In [7]:
result=(
    CLIENT.query
        .get(class_name="Work", properties=["identifier", "title"])
        .with_near_vector({"vector": vector})
        .with_limit(100)
        .do()
)
jprint(result)

<IPython.core.display.JSON object>

In [13]:
result=(
    CLIENT.query
        .get(class_name="Work", properties=["collection", "identifier", "title"])
        .with_limit(100)
        .with_where({
            "path": ["collection"],
            "operator": "NotEqual",
            "valueText": work["collection"]
        })
        .with_near_vector({"vector": vector})
        .do()
)
jprint(result)

<IPython.core.display.JSON object>

In [12]:
CLIENT.schema.get("Work")["properties"]

[{'dataType': ['text'],
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-transformers': {'skip': False,
    'vectorizePropertyName': False}},
  'name': 'title',
  'tokenization': 'word'},
 {'dataType': ['text'],
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-transformers': {'skip': False,
    'vectorizePropertyName': False}},
  'name': 'accession_number',
  'tokenization': 'word'},
 {'dataType': ['text[]'],
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-transformers': {'skip': False,
    'vectorizePropertyName': False}},
  'name': 'alternate_title',
  'tokenization': 'word'},
 {'dataType': ['text'],
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-transformers': {'skip': False,
    'vectorizePropertyName': False}},
  'name': 'api_model',
  'tokenization': 'word'},
 {'dataType': ['text'],
  'indexFilterable': True,
  'indexSearchable': True,
  'modu